# Tissue

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Uberon](http://obophenotype.github.io/uberon)

Here we show how to access and search Tissue ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-tissue --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
tissues = lb.Tissue.public(organism="all")
tissues

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = tissues.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = tissues.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.alveolus_of_lung

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["alveolus of lung"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = tissues.lookup(tissues.ontology_id)

In [ ]:
lookup.uberon_0000031

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
tissues.search("alveolus lung").head(3)

By default, search also covers synonyms:

In [ ]:
tissues.search("nasal sac").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
tissues.search("nasal sac", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
tissues.search(
    "alveolus in the lung",
    field=tissues.definition,
).head()

## Standardize Tissue identifiers

Let us generate a `DataFrame` that stores a number of Tissue identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "UBERON:0000000"
        "UBERON:0000005"
        "UBERON:0000001"
        "UBERON:0000002"
        "This tissue does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = tissues.validate(df_orig.index, tissues.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="Tissue").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="uberon", version="2023-04-19", organism="all"
).one()
tissues= lb.Tissue.public(public_source=public_source)
tissues

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-tissue
!rm -r test-tissue